In [70]:
import pprint
import numpy as np
import pandas as pd
import scipy.linalg as sl
from sklearn.linear_model import LinearRegression
from scipy.linalg import cho_factor, cho_solve

## Problem 1

#### 1.1 Read the data and include an intercept in your model

In [16]:
# Read the data by using pandas
longley = pd.read_csv("longley.dat", header=None, delim_whitespace=True)

In [66]:
# Rename the column 
longley.rename(columns={0:"number of people employed", 1:"GNP implicit price deflator", 2: "GNP", 3:"number of unemployed",
4: "number of people in the armed forces", 5: "noninst. pop", 6: "year"}, inplace=True)

longley.head()


,number of people employed,GNP implicit price deflator,GNP,number of unemployed,number of people in the armed forces,noninst. pop,year
0,60323,83.0,234289,2356,1590,107608,1947
1,61122,88.5,259426,2325,1456,108632,1948
2,60171,88.2,258054,3682,1616,109773,1949
3,61187,89.5,284599,3351,1650,110929,1950
4,63221,96.2,328975,2099,3099,112075,1951


In [97]:
lm = LinearRegression()

model = lm.fit(longley.iloc[:,0:1], longley.iloc[:,1:])

model.coef_
model.intercept_

print(np.array_str(model.intercept_, precision=1, suppress_small=True))

[     -93.2 -1430482.3    -5539.9    -3312.2    -6824.      1868.5]


### 1.2

In [90]:
A = np.array(longley.iloc[:,1:])
b = np.array(longley.iloc[:,0:1])

P, L, U = scipy.linalg.lu(A)


# print(np.array_str(L, precision=1, suppress_small=True))
# print(np.array_str(U, precision=1, suppress_small=True))

print(np.array_str(P, precision=1, suppress_small=True))

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [92]:
print(np.array_str(np.dot(L,U), precision=1, suppress_small=True))

[[   116.9 554894.    4007.    2827.  130081.    1962. ]
 [    88.5 259426.    2325.    1456.  108632.    1948. ]
 [    88.2 258054.    3682.    1616.  109773.    1949. ]
 [    98.1 346999.    1932.    3594.  113270.    1952. ]
 [   110.8 444546.    4681.    2637.  121950.    1958. ]
 [   112.6 482704.    3813.    2552.  123366.    1959. ]
 [    99.  365385.    1870.    3547.  115094.    1953. ]
 [   100.  363112.    3578.    3350.  116219.    1954. ]
 [   101.2 397469.    2904.    3048.  117388.    1955. ]
 [   104.6 419180.    2822.    2857.  118734.    1956. ]
 [   108.4 442769.    2936.    2798.  120445.    1957. ]
 [    96.2 328975.    2099.    3099.  112075.    1951. ]
 [    89.5 284599.    3351.    1650.  110929.    1950. ]
 [   114.2 502601.    3931.    2514.  125368.    1960. ]
 [   115.7 518173.    4806.    2572.  127852.    1961. ]
 [    83.  234289.    2356.    1590.  107608.    1947. ]]


In [95]:
A_pinv = np.linalg.pinv(A)
x = np.dot(A_pinv, b)

In [96]:
x

array([[-52.99357014],
       [  0.0710732 ],
       [ -0.42346586],
       [ -0.57256867],
       [ -0.41420359],
       [ 48.41786562]])

In [98]:
np.linalg.solve(A, b)

LinAlgError: Last 2 dimensions of the array must be square